In [5]:
%session_id_prefix native-iceberg-dataframe-02
%glue_version 3.0
%idle_timeout 60
%%configure 
{
  "--conf": "spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
  "--datalake-formats": "iceberg",
  "--job-bookmark-option": "job-bookmark-enable",
  "--JOB_NAME": "glue_notebook_cdc_upsert_iceberg"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.4 
Setting session ID prefix to native-iceberg-dataframe-02
Setting Glue version to: 3.0
Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
The following configurations have been updated: {'--conf': 'spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions', '--datalake-formats': 'iceberg', '--job-bookmark-option': 'job-bookmark-enable', '--JOB_NAME': 'glue_notebook_cdc_upsert_iceberg'}


In [1]:
from awsglue.job import Job

Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Session ID: 78c45835-6a4d-428a-a08e-ed42a8a7b71e
Applying the following default arguments:
--glue_kernel_version 1.0.4
--enable-glue-datacatalog true
--conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
--datalake-formats iceberg
--job-bookmark-option job-bookmark-enable
--JOB_NAME glue_notebook_cdc_upsert_iceberg
Waiting for session 78c45835-6a4d-428a-a08e-ed42a8a7b71e to get into ready status...
Session 78c45835-6a4d-428a-a08e-ed42a8a7b71e has been created.



In [2]:
catalog_name = "glue_catalog"
bucket_name = "chiholee-datalake001"
database_name = "ecommerce"

source_bucket_prefix = "transaction/cdc/raw"
source_path = f"s3://{bucket_name}/{source_bucket_prefix}"
source_table_name = "orders"

iceberg_bucket_prefix = "transaction/iceberg"
warehouse_path = f"s3://{bucket_name}/{iceberg_bucket_prefix}"
iceberg_table_name = "orders_cdc_iceberg"





In [3]:
pk = 'order_id'

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.warehouse", f"{warehouse_path}") \
    .config(f"spark.sql.catalog.{catalog_name}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

In [5]:
import sys
from awsglue.context import GlueContext
from awsglue.utils import getResolvedOptions


glueContext = GlueContext(spark)

args = getResolvedOptions(sys.argv, ['JOB_NAME'])
job = Job(glueContext)
job.init(args['JOB_NAME'], args)


In [ ]:
cdcDyf = glueContext.create_dynamic_frame_from_options(
    connection_type='s3',
    connection_options={
        'paths': [f'{source_path}/{database_name}/{source_table_name}/'],
        'groupFiles': 'none',
        'recurse': True
    },
    format='parquet',
    transformation_ctx='cdcDyf')

In [ ]:
print(f"## Count of CDC data after last job bookmark:{cdcDyf.count()}")

In [ ]:
cdcDf = cdcDyf.toDF()

In [22]:
# cdcDf.show()

+---+-------------------+--------+--------+---------+-----------+-------------------+-----------+----------+
| Op|    dms_update_time|order_id|promo_id|order_cnt|order_price|           order_dt|customer_id|product_id|
+---+-------------------+--------+--------+---------+-----------+-------------------+-----------+----------+
|  I|2024-04-26 12:02:57|  303311| PROMO20|        9|       8000|2024-04-24 22:23:43|         81|         9|
|  I|2024-04-26 12:02:57|  303312| PROMO06|        5|      15000|2024-04-24 22:23:45|         70|        18|
|  I|2024-04-26 12:02:57|  303313| PROMO02|        6|      50000|2024-04-24 22:23:45|         84|        15|
|  I|2024-04-26 12:02:57|  303314| PROMO06|        3|      31000|2024-04-24 22:23:47|          3|         2|
|  U|2024-04-26 12:02:57|  237501| PROMO20|        9|      29000|2024-04-24 22:23:49|         88|         1|
|  I|2024-04-26 12:02:57|  303315| PROMO12|        8|      18000|2024-04-24 22:23:49|         81|         1|
|  I|2024-04-26 12:

In [55]:
import sys
from pyspark.sql import Window
from pyspark.sql import functions as F 

In [90]:
cdcDf.createOrReplaceTempView("cdcDf")

In [91]:
cdcDf = spark.sql("""
select *
from cdcDf
where (order_id, order_dt) in
(
    select order_id, max(order_dt) max_op_time
    from cdcDf
    group by order_id
)
"""
)

In [ ]:
cdcInsertCount = cdcDf.filter("Op = 'I'").count()
cdcUpdateCount = cdcDf.filter("Op = 'U'").count()
cdcDeleteCount = cdcDf.filter("Op = 'D'").count()
print(f"Inserted count: {cdcInsertCount}")
print(f"Updated count: {cdcUpdateCount}")
print(f"Deleted count: {cdcDeleteCount}")
print(f"Total CDC count: {cdcDf.count()}")

In [94]:
dropColumnList = ['Op','dms_update_time']

In [95]:
from datetime import datetime
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import concat, col, lit, to_timestamp

current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
cdcDf = cdcDf.withColumn('order_dt',to_timestamp(col('order_dt')))
cdcDf = (cdcDf
      .withColumn('year', year(col('order_dt')))
      .withColumn('month', month(col('order_dt')))
      .withColumn('day', dayofmonth(col('order_dt')))
     )
cdcDf = cdcDf.withColumn('last_applied_date',to_timestamp(lit(current_datetime)))



In [96]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog_name}.{database_name}")
existing_tables = spark.sql(f"SHOW TABLES IN {catalog_name}.{database_name};")
df_existing_tables = existing_tables.select('tableName').rdd.flatMap(lambda x:x).collect()

In [97]:
upsertDf = cdcDf.filter("Op != 'D'").drop(*dropColumnList)
upsertDf.createOrReplaceTempView(f"{source_table_name}_upsert")

In [98]:
# upsertDf.show()

+--------+--------+---------+-----------+-------------------+-----------+----------+----+-----+---+-------------------+
|order_id|promo_id|order_cnt|order_price|           order_dt|customer_id|product_id|year|month|day|  last_applied_date|
+--------+--------+---------+-----------+-------------------+-----------+----------+----+-----+---+-------------------+
|  303326| PROMO18|        1|      32000|2024-04-24 22:24:00|         81|         4|2024|    4| 24|2024-04-26 12:54:54|
|  303334| PROMO15|        7|      30000|2024-04-24 22:24:11|         34|        10|2024|    4| 24|2024-04-26 12:54:54|
|   62958| PROMO12|        5|      24000|2024-04-24 22:24:15|         24|         9|2024|    4| 24|2024-04-26 12:54:54|
|  303339| PROMO14|        5|      29000|2024-04-24 22:24:21|         70|         6|2024|    4| 24|2024-04-26 12:54:54|
|  303348| PROMO11|        5|      35000|2024-04-24 22:24:38|         66|         7|2024|    4| 24|2024-04-26 12:54:54|
|  303350| PROMO06|        9|       9000

In [99]:
# spark.sql(f"""
# select order_id, count(*)
# from {catalog_name}.{database_name}.{iceberg_table_name}
# group by order_id
# having count(*) > 1
# """).show()

+--------+--------+
|order_id|count(1)|
+--------+--------+
+--------+--------+


In [100]:
# spark.sql(f"""
# select order_id, count(*)
# from {source_table_name}_upsert
# group by order_id
# having count(*) > 1
# """).show()

+--------+--------+
|order_id|count(1)|
+--------+--------+
+--------+--------+


In [101]:
deleteDf = cdcDf.filter("Op = 'D'").drop(*dropColumnList)
deleteDf.createOrReplaceTempView(f"{source_table_name}_delete")

In [102]:
# deleteDf.show()

+--------+--------+---------+-----------+--------+-----------+----------+----+-----+---+-----------------+
|order_id|promo_id|order_cnt|order_price|order_dt|customer_id|product_id|year|month|day|last_applied_date|
+--------+--------+---------+-----------+--------+-----------+----------+----+-----+---+-----------------+
+--------+--------+---------+-----------+--------+-----------+----------+----+-----+---+-----------------+


In [103]:
print(f"Table {source_table_name}_iceberg is upserting...")
spark.sql(f"""MERGE INTO {catalog_name}.{database_name}.{iceberg_table_name} t
    USING {source_table_name}_upsert s ON s.{pk} = t.{pk}
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *
    """)

Table orders_iceberg is upserting...
DataFrame[]


In [105]:
spark.sql(f"""
select min(order_dt), max(order_dt)
from {catalog_name}.{database_name}.{iceberg_table_name}
""").show()


+-------------------+-------------------+
|      min(order_dt)|      max(order_dt)|
+-------------------+-------------------+
|2024-04-18 01:42:43|2024-04-26 11:59:30|
+-------------------+-------------------+
